# Data-Encryption-Standard (DES) with OpenLane

```
SPDX-License-Identifier: Apache-2.0
```

|Name|Affiliation| Email |IEEE Member|SSCS Member|
|:--:|:----------:|:----------:|:----------:|:----------:|
|Jacob Mack|University of Michigan|jmackmi@umich.edu|Yes|Yes|




## Introduction


---



This notebook introduces the Data Encryption Standard (DES), demonstrates DES operation, and implements a DES design thru the [OpenLane](https://github.com/The-OpenROAD-Project/OpenLane/) GDS to RTL flow targeting the [open source SKY130 PDK](https://github.com/google/skywater-pdk/).

This notebook is broken down into answering three questions:
- What is DES and how is it implemented?
- How to run a RTL and C++ simulation of DES and how do the results compare?
- How to run the OpenLane Flow and generate design metrics?

### DES


---


DES stands for **D**ata **E**ncryption **S**tandard and is a symmetric-key algorithm for the encryption of digital data. DES uses the same key for both encryption and decryption which makes it a symmetric-key algorithm. DES was developed in early 1970s by IBM and was published as an official Federal Information Processing Standard for the United States in 1977.
</br></br>
The input to be encrypted is called the **plain text** and the output scrambled text is called the **cipher text**.
</br></br>
For our example, the plain text is the hexidecimal number 01234567890ABCDEF and the **key** is the hexadecimal number 133457799BBCDFF1.
</br></br>
The three main components of DES are **Key Generation**, **Encryption of Plain Text**, and **Decryption of Cipher Text**.

#### Key Generation


---


16 sub-keys are created from the original key for the 16 rounds of encryption of plain text.
</br></br>
Starting with the original key, 133457799BBCDFF1, it is permutated according to the Permutated Choice 1 (PC-1):

![PC1](https://github.com/jmack2201/code-a-chip-des/blob/pipeline/img/PC1.png?raw=true)

Before PC-1:

Key: 133457799BBCDFF1

After PC-1:

Key: F0CCAAF556678F

During PC-1, some bits of the key are not used (8,16,24,32,40,48,56,64) and are used as parity bits.
</br></br>
The permutated key is then divided into left and right halves:

Left: FOCCAAF

Right: AACCF1E

Depending on what round the sub-key is generated for, the left and right halves will circular left shift either by one or two:

| Round Number      | Shift Amount |
| ----------- | ----------- |
| 1,2,8,16      | 1       |
| 4,5,6,7,9,10,11,12,13,14,15   | 2       |

The sub-key halves for round one will circle shift by one:

Left: E19955F

Right: AACCF1E
</br></br>
Subsequent sub-keys are based on the previous sub-keys such that the sub-key for round two is derived from round one and the sub-key for round three is derived from round two etc.
</br></br>
For round two, it takes the left and right halves of the previous round (one) and circle left shifts according to shift amount table:


Left: C332ABF

Right: 5599E3D
</br></br>
This process is repeated for all 16 rounds and this table shows the result of that process:

| Round Number      | Left | Right |
| ----------- | ----------- | -----------|
| 1      |   E19955F     |AACCF1E |
| 2      |   C332ABF     |5599E3D |
| 3      |    0CCAAFF    |56678F5 |
| 4      |      332ABFC  |599E3D5 |
| 5     |   CCAAFF0     |6678F55 |
| 6      |    32ABFC3    |99E3D55 |
| 7      |    CAAFF0C    | 678F556|
| 8      |    2ABFC33    |9E3D559 |
| 9      |    557F866    | 3C7AAB3|
| 10      |    55FE199    |F1EAACC |
| 11      |   57F8665     |C7AAB33 |
| 12      |   5FE1995     |1EAACCF |
| 13      |  7F86655      |7AAB33C |
| 14      |   FE19955     |EAACCF1 |
| 15      |    F866557    |AAB33C7 |
| 16      |    F0CCAAF    |556678F |

Before the sub-keys are sent to be used in the encryption of the plain text, each sub-key left and right half is combined and then permutated according to Permutated Choice 2 (PC-2):


![PC2](https://github.com/jmack2201/code-a-chip-des/blob/pipeline/img/PC2.png?raw=true)


This is the result of PC-2 on the combined left and right halves for round one:

Before: E19955FAACCF1E

After: 1B02EFFC7072
</br></br>
The final sub-keys for each round after PC-2 are:

| Round Number      | Sub-Key |
| ----------- | ----------- |
| 1      |    1B02EFFC7072    |
| 2      |    79AED9DBC9E5    |
| 3      |    55FC8A42CF99    |
| 4      |    72ADD6DB351D    |
| 5     |     7CEC07EB53A8   |
| 6      |    63A53E507B2F    |
| 7      |    EC84B7F618BC    |
| 8      |     F78A3AC13BFB   |
| 9      |    E0DBEBEDE781    |
| 10      |    B1F347BA464F    |
| 11      |     215FD3DED386   |
| 12      |    7571F59467E9    |
| 13      |    97C5D1FABA41    |
| 14      |    5F43B7F2E73A    |
| 15      |  BF918D3D3F0A      |
| 16      |   CB3D8B0E17F5     |



#### Encryption of Plain Text


---


To start the encryption, the plain text goes through an inital permutation:

![IP](https://github.com/jmack2201/code-a-chip-des/blob/pipeline/img/IP.png?raw=true)

Before: 0123456789ABCDEF

After: CC00CCFFF0AAF0AA

</br>



The output of the initial permutation is divided into left and right halves:

Left: CC00CCFF

Right: F0AAF0AA

</br>


A round of encryption's output is the following left and right halves:

Left: Previous round's right half

Right: Previous round's left half XOR fiestel(Previous round's right half, Round Sub-Key)

(For the first round, the previous left and right halves come from the initial permutation.)

</br>




To compute the fiestel function, the previous round's right half is first expanded according to this permutation: ![E](https://github.com/jmack2201/code-a-chip-des/blob/pipeline/img/E.png?raw=true)


Before: F0AAF0AA

After: 7A15557A1555

</br>

Next this expanded value is XORed with the round sub-key (for round one, the sub-key is 1B02EFFC7072):

7A15557A1555 XOR 1B02EFFC7072 = 6117BA866527

</br>

After XORing with the sub-key, the value is divided into eight 6-bit slices (1st, 2nd, 3rd, ... 8th 6-bit slice) for S-Box subsitition:
011000,
010001,
011110,
111010,
100001,
100110,
010100,
100111.

</br>

An S-Box is a look up table that contains a 4-bit value and is indexed by the following:

Row = Outer Two Bits, Example: 100001 -> 11

Column = Inner Four Bits, Example: 100001 -> 0000

</br>

There are eight S-Boxes corresponding to the eight 6-bit slices and each S-Box has different values from each other. The eight S-Boxes are:

S-Box 1

| Row/Column | 0  | 1  | 2  | 3 | 4  | 5  | 6  | 7  | 8  | 9  | A  | B  | C  | D  | E | F  |
|------------|----|----|----|---|----|----|----|----|----|----|----|----|----|----|---|----|
| 0          | 14 | 4  | 13 | 1 | 2  | 15 | 11 | 8  | 3  | 10 | 6  | 12 | 5  | 9  | 0 | 7  |
| 1          | 0  | 15 | 7  | 4 | 14 | 2  | 13 | 1  | 10 | 6  | 12 | 11 | 9  | 5  | 3 | 8  |
| 2          | 4  | 1  | 14 | 8 | 13 | 6  | 2  | 11 | 15 | 12 | 9  | 7  | 3  | 10 | 5 | 0  |
| 3          | 15 | 12 | 8  | 2 | 4  | 9  | 1  | 7  | 5  | 11 | 3  | 14 | 10 | 0  | 6 | 13 |

S-Box 2

| Row/Column | 0  | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9 | A  | B  | C  | D | E  | F  |
|------------|----|----|----|----|----|----|----|----|----|---|----|----|----|---|----|----|
| 0          | 15 | 1  | 8  | 14 | 6  | 11 | 3  | 4  | 9  | 7 | 2  | 13 | 12 | 0 | 5  | 10 |
| 1          | 3  | 13 | 4  | 7  | 15 | 2  | 8  | 14 | 12 | 0 | 1  | 10 | 6  | 9 | 11 | 5  |
| 2          | 0  | 14 | 7  | 11 | 10 | 4  | 13 | 1  | 5  | 8 | 12 | 6  | 9  | 3 | 2  | 15 |
| 3          | 13 | 8  | 10 | 1  | 3  | 15 | 4  | 2  | 11 | 6 | 7  | 12 | 0  | 5 | 14 | 9  |

S-Box 3

| Row/Column | 0  | 1  | 2  | 3  | 4 | 5  | 6  | 7  | 8  | 9  | A  | B  | C  | D  | E  | F  |
|------------|----|----|----|----|---|----|----|----|----|----|----|----|----|----|----|----|
| 0          | 10 | 0  | 9  | 14 | 6 | 3  | 15 | 5  | 1  | 13 | 12 | 7  | 11 | 4  | 2  | 8  |
| 1          | 13 | 7  | 0  | 9  | 3 | 4  | 6  | 10 | 2  | 8  | 5  | 14 | 12 | 11 | 15 | 1  |
| 2          | 13 | 6  | 4  | 9  | 8 | 15 | 3  | 0  | 11 | 1  | 2  | 12 | 5  | 10 | 14 | 7  |
| 3          | 1  | 10 | 13 | 0  | 6 | 9  | 8  | 7  | 4  | 15 | 14 | 3  | 11 | 5  | 2  | 12 |

S-Box 4

| Row/Column | 0  | 1  | 2  | 3 | 4  | 5  | 6  | 7  | 8  | 9 | A | B  | C  | D  | E  | F  |
|------------|----|----|----|---|----|----|----|----|----|---|---|----|----|----|----|----|
| 0          | 7  | 13 | 14 | 3 | 0  | 6  | 9  | 10 | 1  | 2 | 8 | 5  | 11 | 12 | 4  | 15 |
| 1          | 13 | 8  | 11 | 5 | 6  | 15 | 0  | 3  | 4  | 7 | 2 | 12 | 1  | 10 | 14 | 9  |
| 2          | 10 | 6  | 9  | 0 | 12 | 11 | 7  | 13 | 15 | 1 | 3 | 14 | 5  | 2  | 8  | 4  |
| 3          | 3  | 15 | 0  | 6 | 10 | 1  | 13 | 8  | 9  | 4 | 5 | 11 | 12 | 7  | 2  | 14 |

S-Box 5

| Row/Column | 0  | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | A  | B  | C  | D | E  | F  |
|------------|----|----|----|----|----|----|----|----|----|----|----|----|----|---|----|----|
| 0          | 2  | 12 | 4  | 1  | 7  | 10 | 11 | 6  | 8  | 5  | 3  | 15 | 13 | 0 | 14 | 9  |
| 1          | 14 | 11 | 2  | 12 | 4  | 7  | 13 | 1  | 5  | 0  | 15 | 10 | 3  | 9 | 8  | 6  |
| 2          | 4  | 2  | 1  | 11 | 10 | 13 | 7  | 8  | 15 | 9  | 12 | 5  | 6  | 3 | 0  | 14 |
| 3          | 11 | 8  | 12 | 7  | 1  | 14 | 2  | 13 | 6  | 15 | 0  | 9  | 10 | 4 | 5  | 3  |

S-Box 6

| Row/Column | 0  | 1  | 2  | 3  | 4 | 5  | 6  | 7  | 8  | 9  | A  | B  | C  | D  | E  | F  |
|------------|----|----|----|----|---|----|----|----|----|----|----|----|----|----|----|----|
| 0          | 12 | 1  | 10 | 15 | 9 | 2  | 6  | 8  | 0  | 13 | 3  | 4  | 14 | 7  | 5  | 11 |
| 1          | 10 | 15 | 4  | 2  | 7 | 12 | 9  | 5  | 6  | 1  | 13 | 14 | 0  | 11 | 3  | 8  |
| 2          | 9  | 14 | 15 | 5  | 2 | 8  | 12 | 3  | 7  | 0  | 4  | 10 | 1  | 13 | 11 | 6  |
| 3          | 4  | 3  | 2  | 12 | 9 | 5  | 15 | 10 | 11 | 14 | 1  | 7  | 6  | 0  | 8  | 13 |

S-Box 7

| Row/Column | 0  | 1  | 2  | 3  | 4  | 5 | 6  | 7  | 8  | 9  | A | B  | C  | D  | E | F  |
|------------|----|----|----|----|----|---|----|----|----|----|---|----|----|----|---|----|
| 0          | 4  | 11 | 2  | 14 | 15 | 0 | 8  | 13 | 3  | 12 | 9 | 7  | 5  | 10 | 6 | 1  |
| 1          | 13 | 0  | 11 | 7  | 4  | 9 | 1  | 10 | 14 | 3  | 5 | 12 | 2  | 15 | 8 | 6  |
| 2          | 1  | 4  | 11 | 13 | 12 | 3 | 7  | 14 | 10 | 15 | 6 | 8  | 0  | 5  | 9 | 2  |
| 3          | 6  | 11 | 13 | 8  | 1  | 4 | 10 | 7  | 9  | 5  | 0 | 15 | 14 | 2  | 3 | 12 |

S-Box 8

| Row/Column | 0  | 1  | 2  | 3 | 4  | 5  | 6  | 7  | 8  | 9  | A  | B  | C  | D  | E  | F  |
|------------|----|----|----|---|----|----|----|----|----|----|----|----|----|----|----|----|
| 0          | 13 | 2  | 8  | 4 | 6  | 15 | 11 | 1  | 10 | 9  | 3  | 14 | 5  | 0  | 12 | 7  |
| 1          | 1  | 15 | 13 | 8 | 10 | 3  | 7  | 4  | 12 | 5  | 6  | 11 | 0  | 14 | 9  | 2  |
| 2          | 7  | 11 | 4  | 1 | 9  | 12 | 14 | 2  | 0  | 6  | 10 | 13 | 15 | 3  | 5  | 8  |
| 3          | 2  | 1  | 14 | 7 | 4  | 10 | 8  | 13 | 15 | 12 | 9  | 0  | 3  | 5  | 6  | 11 |

</br>

Following the example, here are the values (in hexadecimal) from each S-Box based on the eight 6-bit slices:

| S-Box      | 6-Bit Slice | Row | Column | Value |
| ----------- | ----------- | ----------- | ----------- |-----------|
| 1      |     011000   | 0 | C | 5 |
| 2      |    010001    | 1 | 8 | C |
| 3      |    011110    | 0 | F |  8|
| 4      |   111010     | 2 | D | 2 |
| 5     |   100001     |  3|  0|  B|
| 6      |    100110    | 2 | 3 | 5 |
| 7      |    010100    | 0 | A | 9 |
| 8      |    100111    | 3 | 3 | 7 |

The outputs of the S-Boxes are then combined to form a 32-bit value: 5C82B597

</br>


Following the S-Box subsititution, the output is permutated again according to this permutation: ![P](https://github.com/jmack2201/code-a-chip-des/blob/pipeline/img/P.png?raw=true)

Before: 5C82B597

After: 234AA9BB

</br>

Following the permutation after the S-Box subsitition, the value of the feistel function for round one is found and the output round right value can be found:

CC00CCFF XOR 234AA9BB = EF4A6544

</br>

After round one, the output left and right values are:

Left: F0AAF0AA

Right: EF4A6544

</br>

This round computation repeats 15 more times and uses the appropriate sub-keys for each round. The output left and right for each round are located in the table below:

| Round Number      | Left | Right |
| ----------- | ----------- | -----------|
| 1      |    F0AAF0AA    |EF4A6544 |
| 2      |   EF4A6544     |CC017709 |
| 3      |    CC017709    |A25C0BF4 |
| 4      |    A25C0BF4    | 77220045|
| 5     |   77220045     |8A4FA637 |
| 6      |  8A4FA637      | E967CD69|
| 7      |  E967CD69      | 064ABA10|
| 8      |    064ABA10    | D5694B90|
| 9      |   D5694B90     |247CC67A |
| 10      |   247CC67A     |B7D5D7B2 |
| 11      | B7D5D7B2       |C5783C78 |
| 12      | C5783C78       |75BD1858 |
| 13      |    75BD1858    |18C3155A |
| 14      | 18C3155A       |C28C960D |
| 15      |  C28C960D      |43423234 |
| 16      |    43423234    | 0A4CD995 |

</br>

The final step in finishing the encryption is a final permuation which is the inverse of the initial permuation on the combined left and right halves (there is one more swap of left and right after round 16 before the final permutation):
![invIP](https://github.com/jmack2201/code-a-chip-des/blob/pipeline/img/invIP.png?raw=true)

Before: 0A4CD99543423234

After: 85E813540F0AB405

</br>

For an input plain text of 0123456789ABCDEF and a key of 133457799BBCDFF1, the cipher text 85E813540F0AB405 is generated.



#### Decryption of Cipher Text


---


To decrypt a given cipher text using a key, the processs is similar to encryption except the cipher text is used as the plain text.
</br></br>
In key generation, the sub-keys are created in the same process, but the order of the rounds they are applied to is reversed.
</br></br>
In our example, the sub-key created for round one is 1B02EFFC7072, but the sub-key will be applied for the 16th round.
</br></br>
This table shows the updated order of keys for our example key of 133457799BBCDFF1:

| Round Number      | Sub-Key |
| ----------- | ----------- |
| 1              | CB3D8B0E17F5 |
| 2              | BF918D3D3F0A |
| 3             | 5F43B7F2E73A |
| 4              | 97C5D1FABA41 |
| 5             | 7571F59467E9 |
| 6              | 215FD3DED386 |
| 7              | B1F347BA464F |
| 8              | E0DBEBEDE781 |
| 9              | F78A3AC13BFB |
| 10              | EC84B7F618BC |
| 11              | 63A53E507B2F |
| 12              | 7CEC07EB53A8 |
| 13              | 72ADD6DB351D |
| 14              | 55FC8A42CF99  |
| 15              | 79AED9DBC9E5 |
| 16             | 1B02EFFC7072 |

</br>Decrypting the cipher text generated from the previous encryption example (85E813540F0AB405) using the same key (133457799BBCDFF1) would result in the created cipher text of 0123456789ABCDEF. If the same key is not used during decryption, the plain text is further scrambled which is the basis of [Triple DES](https://en.wikipedia.org/wiki/Triple_DES).



### DES Hardware Implementaton


---



#### Block Diagram

Shown here is a diagram of the DES implementation in hardware with block names corresponding to the Verilog module names:

![updatedBD](https://github.com/jmack2201/code-a-chip-des/blob/pipeline/img/block_diagram.jpg?raw=true)

</br>


#### Configuration

To enable configurability of the DES Hardware, a hardware parameter `NUM_ROUNDS_PER_FF` describes the granularity of pipelining within the design. A value of 0 sets no pipelining within the design and a value of 4 divides the round stack into groups of 4 for pipelining. The diagram below illustrates different confirgurations of DES:

![BD_pipe](https://github.com/jmack2201/code-a-chip-des/blob/pipeline/img/bd_pipeline.jpg?raw=true)


Depending on the target area and timing specification, `NUM_ROUNDS_PER_FF` can be tuned to meet those requirements.

## Simulation


---



In [2]:
#@title Download Dependencies

#@markdown The main package that will be used to test the Verilog implemention is [Icarus Verilog](https://steveicarus.github.io/iverilog/index.html) (iverilog).

#@markdown iverilog is a free and open-source tool used to compile and to simulate Verilog file(s).
!apt-get install iverilog


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  gtkwave
The following NEW packages will be installed:
  iverilog
0 upgraded, 1 newly installed, 0 to remove and 10 not upgraded.
Need to get 2,130 kB of archives.
After this operation, 6,749 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 iverilog amd64 11.0-1.1 [2,130 kB]
Fetched 2,130 kB in 0s (5,476 kB/s)
Selecting previously unselected package iverilog.
(Reading database ... 120880 files and directories currently installed.)
Preparing to unpack .../iverilog_11.0-1.1_amd64.deb ...
Unpacking iverilog (11.0-1.1) ...
Setting up iverilog (11.0-1.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
#@title Download Verilog and C++ Files

#@markdown The verilog and C++ files are provided by cloning a GitHub repo into a folder \"code-a-chip-des". Each file in the \"code-a-chip-des/scripts", \"code-a-chip-des/verilog/src", and \"code-a-chip-des/tb" sub-directories is described below:

#@markdown * code-a-chip-des/scripts
#@markdown  * `Makefile`: Handles compilation and simulation for both C++ and Verilog implementations of DES.
#@markdown  * `des.cpp`: Describles C++ implemention of DES for both encryption and decryption.
#@markdown * code-a-chip-des/verilog/src
#@markdown  * `des_config.v`: Configures the amount of rounds per pipeline register.
#@markdown  * `des_top.v`: Serves as the top level module, instantiates `init_perm`, `key_gen`,`round_stack`, and `final_perm`, and creates the input and output flip-flop.
#@markdown  * `feistel_function.v`: Instantiates `p_box_32_48`, `s_box_48_32`, and `p_box_32_32`, and links them together.
#@markdown  * `final_perm.v`: Performs the inverse of the Inital Permuation after applying 16 rounds of the DES algorithm to the plain text.
#@markdown  * `init_perm.v`: Performs the Initial Permutation of the plain text.
#@markdown  * `key_gen.v`: Creates 16 copies of `round_key_gen` to create the 16 total round keys.
#@markdown  * `key_mux.v`: Multiplexes the output of `key_gen` based on operation mode.
#@markdown  * `p_box_32_32.v`: Permutates the output of `s_box_48_32` for the `feistel_function` output.
#@markdown  * `p_box_32_48.v`: Expands the input of `feistel_function` to be XORed with the round key.
#@markdown  * `p_box_56_48.v`: Compresses the output of `round_key_gen` and connects to the `key_mux` of the assigned round.
#@markdown  * `p_box_64_56.v`: Compresses and permutates the input key before starting generation of 16 round keys.
#@markdown  * `round_ff.v`: Implements the round pipeline register.
#@markdown  * `round_key_gen.v`: Creates a single round key and passes round key to latter round key generation stages
#@markdown  * `round_stack.v`: Constructs 16 `round` instances and prepares input to `final_perm`.
#@markdown  * `round.v`: Instantiates an instance of `feistel_function`, prepares the round outputs, and creates a `round_ff` if desired.
#@markdown  * `s_box_6_4.v`: Describes the eight unique S Boxes used in `feistel_function`.
#@markdown  * `s_box_48_32.v`: Creates eight `s_box_6_4` modules and links them together.
#@markdown * code-a-chip-des/verilog/tb
#@markdown   * `des_top_tasks.v`: Defines the three tasks primarily used for Verilog simulation.
#@markdown   * `tb_des_top.v`: Contains the testbench for the Verilog simulation.



!rm -rf code-a-chip-des/
!git clone -b pipeline https://github.com/jmack2201/code-a-chip-des.git

Cloning into 'code-a-chip-des'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 337 (delta 169), reused 266 (delta 101), pack-reused 0
Receiving objects: 100% (337/337), 50.09 KiB | 2.50 MiB/s, done.
Resolving deltas: 100% (169/169), done.


#### Configure DES

In [4]:
#@markdown To configure the DES, type in the amount of rounds per pipeline register desired in the DES:
#@markdown * NUM_ROUNDS_PER_FF: $\in [ 0, 16 ]$

NUM_ROUNDS_PER_FF = 4 #@param {type: "integer"}

des_config_file = open("/content/code-a-chip-des/verilog/src/des_config.v","w")

des_config_file.write("`ifndef des_config\n")
des_config_file.write("\t`define NUM_ROUND_STAGES {}\n".format(NUM_ROUNDS_PER_FF))
des_config_file.write("`endif\n")

des_config_file.close()

#### Run Simulation

In [8]:
#@markdown Configure the DES simulation by filling in the following:
#@markdown *   Operation: Encryption or Decryption
#@markdown *   Plain Text: 16 Character Hexadecimal Number
#@markdown *   Cipher Key: 16 Character Hexadecimal Number

operation = "Encryption" #@param ["Encryption","Decryption"]
plain_text = "0123456789abcdef" #@param {type:"string"}
cipher_key = "133457799BBCDFF1" #@param {type:"string"}

#print out current operation
print("============================================")
print("Operation: {}\nPlain Text: {}\nCipher Key: {}".format(operation,plain_text.upper(),cipher_key))
print("============================================")

#run golden C++ simulation
print("Running Golden C++ Simulation...")
!cd /content/code-a-chip-des/scripts/; make cpp_flow plain_text={plain_text} cipher_key={cipher_key} encrypt_decrypt={operation} > make_cpp.log
#pull result cipher text from cpp log file
golden_sim_out = ""
golden_sim_file = open("/content/code-a-chip-des/scripts/make_cpp.log","r")
golden_sim_file_lines = golden_sim_file.readlines()
golden_sim_file.close()
for line in golden_sim_file_lines:
  if "Cipher Text" in line:
    golden_sim_out = line.split(" ")[len(line.split(" "))-1].replace("\n", "").upper()
    break
print("Golden C++ Simulation Output Cipher Text: {}".format(golden_sim_out))
print("Golden C++ Simulation Complete")
print("============================================")

#run verilog simulation
print("Running Verilog Simulation...")
!cd /content/code-a-chip-des/sim/; make verilog plain_text={plain_text} cipher_key={cipher_key} encrypt_decrypt={operation} > make_verilog.log
#pull result cipher text from verilog log file
verilog_sim_out = ""
verilog_sim_file = open("/content/code-a-chip-des/sim/make_verilog.log","r")
verilog_sim_file_lines = verilog_sim_file.readlines()
verilog_sim_file.close()
for line in verilog_sim_file_lines:
  if "Valid: 1" in line:
    verilog_sim_out = line.split("\t")[2].split(" ")[2].upper()
    break
print("Verilog Simulation Output Cipher Text: {}".format(verilog_sim_out))
print("Verilog Simulation Complete")
print("============================================")
if (verilog_sim_out == golden_sim_out):
    print("The Verilog and C++ Simulation are the same!")
    print("============================================")


Operation: Encryption
Plain Text: 0123456789ABCDEF
Cipher Key: 133457799BBCDFF1
Running Golden C++ Simulation...
Golden C++ Simulation Output Cipher Text: 85E813540F0AB405
Golden C++ Simulation Complete
Running Verilog Simulation...
Verilog Simulation Output Cipher Text: 85E813540F0AB405
Verilog Simulation Complete
The Verilog and C++ Simulation are the same!


## OpenLane Flow


---



In [7]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

openlane_version = 'latest' #@param {type:"string"}
open_pdks_version = 'latest' #@param {type:"string"}

if openlane_version == 'latest':
  openlane_version = ''
if open_pdks_version == 'latest':
  open_pdks_version = ''

import os
import pathlib

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version}
!python -m pip install gdstk gdstk
!python -m pip install libparse libparse
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}


Empty environment created at prefix: /content/conda-env

Pinned packages:
  - python 3.7*


Transaction

  Prefix: /content/conda-env

  Updating specs:

   - openlane=*
   - open_pdks.sky130a=*


  Package                                                Version  Build                 Channel         Size
──────────────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────────────────────────────────

  + open_pdks.sky130a                         1.0.458_0_g8c68aca  20231104_052339       litex-hub     Cached
  + _libgcc_mutex                                            0.1  main                  main          Cached
  + libstdcxx-ng                                          11.2.0  h1234567_1            main          Cached
  + ld_impl_linux-64                                        2.38  h1181459_1            main          Cached
  + ca-certificates     

### Configure Flow

For the target design, a frequency of 16.667 MHz and an area of 600um x 600um are both specified. These values can be changed based on the user's requirements and the [documentation](https://openlane.readthedocs.io/en/latest/reference/configuration.html) can be referenced for more information.

In [16]:
%%writefile config.json
{
    "DESIGN_NAME": "des_top",
    "VERILOG_FILES": "dir::code-a-chip-des/verilog/src/*.v",
    "FP_CORE_UTIL": "50",
    "CLOCK_PORT": "clk",
    "CLOCK_PERIOD": 60,
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 600 600"
}

Overwriting config.json


### Run Flow

[OpenLane](https://openlane.readthedocs.io/en/latest/) is an automated [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) to [GDSII](https://en.wikipedia.org/wiki/GDSII) flow based on several components including [OpenROAD](https://theopenroadproject.org/), [Yosys](https://yosyshq.net/yosys/), [Magic](http://www.opencircuitdesign.com/magic/), [Netgen](http://opencircuitdesign.com/netgen/) and custom methodology scripts for design exploration and optimization targeting [open source PDKs](https://github.com/google/open-source-pdks).

![img](https://openlane.readthedocs.io/en/latest/_images/flow_v1.png)

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

env: PDK=sky130A
OpenLane 2023.11.03_0_gf4f8dad8-conda
All rights reserved. (c) 2020-2022 Efabless Corporation and contributors.
Available under the Apache License, version 2.0. See the LICENSE file for more details.

[INFO]: Using configuration in 'config.json'...
[INFO]: PDK Root: /content/conda-env/share/pdk
[INFO]: Process Design Kit: sky130A
[INFO]: Standard Cell Library: sky130_fd_sc_hd
[INFO]: Optimization Standard Cell Library: sky130_fd_sc_hd
[INFO]: Run Directory: /content/runs/RUN_2023.11.27_06.08.46
[INFO]: Saving runtime environment...
[INFO]: Preparing LEF files for the nom corner...
[INFO]: Preparing LEF files for the min corner...
[INFO]: Preparing LEF files for the max corner...
[WARNING]: PNR_SDC_FILE is not set. It is recommended to write a custom SDC file for the design. Defaulting to BASE_SDC_FILE
[WARNING]: SIGNOFF_SDC_FILE is not set. It is recommended to write a custom SDC file for the design. Defaulting to BASE_SDC_FILE
[INFO]: Running linter (Verilator) (log: 

### Display Layout

In [ ]:
import pathlib
import gdstk
import IPython.display

gdss = sorted(pathlib.Path('runs').glob('*/results/final/gds/*.gds'))
library = gdstk.read_gds(gdss[-1])
top_cells = library.top_level()
top_cells[0].write_svg('des.svg')
IPython.display.SVG('des.svg')

### Metrics

[Documentation](https://openlane.readthedocs.io/en/latest/reference/datapoint_definitions.html)


In [ ]:
import pandas as pd
import pathlib

pd.options.display.max_rows = None
reports = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(reports[-1])
df.transpose()

,0
design,/content
design_name,des_top
config,RUN_2023.11.15_22.05.38
flow_status,flow completed
total_runtime,1h50m53s0ms
routed_runtime,1h9m29s0ms
(Cell/mm^2)/Core_Util,101979.562837
DIEAREA_mm^2,0.34301
CellPer_mm^2,40791.825135
OpenDP_Util,41.01
